In [1]:
import sqlite3
import pandas as pd

DB_PATH = r"C:\Users\kevib\dev\DA\team\MIMIC4-hosp-icu.db"
conn = sqlite3.connect(DB_PATH)

cancer_query = """
SELECT
    ca.subject_id,
    ca.hadm_id
FROM cancer_admissions AS ca
JOIN admissions AS a
  ON ca.subject_id = a.subject_id
 AND ca.hadm_id   = a.hadm_id
JOIN patients AS p
  ON ca.subject_id = p.subject_id
WHERE
    anchor_age >= 18
"""

cancer_admissions = pd.read_sql_query(cancer_query, conn)


In [2]:
#오피오이드 처방 hdam_id 리스트 추출


ndc_mme_table_path = r"C:\Users\kevib\dev\DA\ndc_mme_table.xlsx"
#ndc_mme_table sheet만 불러오기

ndc_mme_df = pd.read_excel(ndc_mme_table_path, sheet_name='NDC - MME', engine='openpyxl', header=None)
#1열 추출, 3행부터
opioid_ndc_digit_codes = ndc_mme_df.iloc[2:, 0].astype(str).tolist()
opioid_query = """
SELECT DISTINCT subject_id, hadm_id, ndc, drug
FROM prescriptions
WHERE
    SUBSTRING(ndc, 1, 9) IN ({})
    """.format(','.join(['?'] * len(opioid_ndc_digit_codes)))
opioid_prescriptions = pd.read_sql_query(opioid_query, conn, params=opioid_ndc_digit_codes)


In [3]:
#오피오이드 환자와 비오피오이드 환자 수 비교
cancer_opioid = pd.merge(cancer_admissions, opioid_prescriptions, on=['subject_id', 'hadm_id'], how='inner')
cancer_non_opioid = pd.merge(cancer_admissions, opioid_prescriptions, on=['subject_id', 'hadm_id'], how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
print(f"Number of cancer admissions with opioid prescriptions: {len(cancer_opioid)}")
print(f"Number of cancer admissions without opioid prescriptions: {len(cancer_non_opioid)}")


Number of cancer admissions with opioid prescriptions: 131653
Number of cancer admissions without opioid prescriptions: 34455


In [4]:
# hadm_id 리스트 추출
hadm_list = cancer_non_opioid["hadm_id"].tolist()

# ? 플레이스홀더 개수 만들기
placeholders = ','.join(['?'] * len(hadm_list))

non_opioid_query = f"""
SELECT DISTINCT subject_id, hadm_id, ndc, drug
FROM prescriptions
WHERE hadm_id IN ({placeholders})
"""

# params 에는 리스트(또는 튜플)를 넘겨야 합니다.
non_opioid_prescriptions = pd.read_sql_query(
    non_opioid_query,
    conn,
    params=hadm_list
)

non_opioid_drug_list = non_opioid_prescriptions["ndc"].tolist()


In [5]:
non_opioid_prescriptions

,subject_id,hadm_id,ndc,drug
0,18327790,20000214,0,Sodium Chloride 0.9% Flush
1,18327790,20000214,50383077933,Lactulose
2,18327790,20000214,60687059501,Famotidine
3,18327790,20000214,61958230101,Vemlidy
4,18327790,20000214,16500008806,Multivitamins W/minerals
...,...,...,...,...
759250,14588438,29999803,68084071301,Venlafaxine XR
759251,14588438,29999803,00378103093,Eplerenone
759252,14588438,29999803,68180051201,Lisinopril
759253,14588438,29999803,50268075615,Torsemide


In [6]:
opioid_prescriptions['drug'].unique()

array(['TraMADOL (Ultram)', 'OxyCODONE (Immediate Release)',
       'OxycoDONE (Immediate Release) ', 'Hydrocodone-Acetaminophen',
       'Oxycodone-Acetaminophen', 'Codeine Sulfate',
       'HYDROmorphone (Dilaudid)', 'Meperidine',
       'Hydrocodone-Acetaminophen (5mg-500mg',
       'Hydrocodone-Acetaminophen (5mg-500mg)',
       'Oxycodone-Acetaminophen (5mg-325mg)',
       'OxycoDONE-Acetaminophen Elixir', 'Fentanyl Citrate',
       'Guaifenesin-CODEINE Phosphate', 'HYDROmorphone', 'TraMADol',
       'OxyCODONE--Acetaminophen (5mg-325mg)', 'Oxycodone SR (OxyconTIN)',
       'NORepinephrine', 'OxyCODONE SR (OxyCONTIN)',
       'OxyCODONE SR (OxyconTIN)', 'Belladonna & Opium (16.2/30mg)',
       'OxycoDONE Liquid', 'Fentanyl Patch', 'OxycoDONE',
       'Diphenoxylate-Atropine', 'HYDROcodone-Acetaminophen (5mg-325mg)',
       'Acetaminophen w/Codeine', 'Hydrocodone-Acetaminophen (5mg-325mg)',
       'Fentanyl PCA', 'Opium Tincture', 'OxyCODONE Liquid',
       'HYDROmorphone-HP', 'Oxy

In [7]:
# 1) 오피오이드가 나온 hadm_id 집합
opioid_hadm_ids = opioid_prescriptions['hadm_id'].unique()

# 2) 그 hadm_id 를 제외한 non_opioid_prescriptions 행만 남기기
non_opioid_only_prescriptions = non_opioid_prescriptions[
    ~non_opioid_prescriptions['hadm_id'].isin(opioid_hadm_ids)
]

# 3) 그 중 hadm_id만 따로 보고 싶으면
non_opioid_hadm = non_opioid_only_prescriptions['hadm_id'].unique()


In [8]:
non_opioid_hadm

array([20000214, 20000588, 20001068, ..., 29998978, 29999326, 29999803],
      shape=(32158,))

In [9]:
residual_drug_list =  non_opioid_only_prescriptions['drug'].unique()

In [10]:
residual_drug_list

array(['Sodium Chloride 0.9%  Flush', 'Lactulose', 'Famotidine', ...,
       'cefOXitin', 'pralsetinib', 'Varicella Virus Vaccine'],
      shape=(3284,), dtype=object)

In [11]:
residual = non_opioid_prescriptions[
    non_opioid_prescriptions['drug'].isin(residual_drug_list)
]

In [12]:
residual['ndc'].unique()

array(['0', '50383077933', '60687059501', ..., '60505076005',
       '00781197501', '00006482700'], shape=(4922,), dtype=object)

In [13]:
import pandas as pd
import sqlite3

import sqlite3
import pandas as pd

DB_PATH = r"C:\Users\kevib\dev\DA\team\MIMIC4-hosp-icu.db"
conn = sqlite3.connect(DB_PATH)
print("\n--- 사후 필터링 및 재분류 시작 ---\n")

# 1. 텍스트 기반 필터링 키워드 정의
# FP_KEYWORDS: 오피오이드 리스트에 있지만 실제로는 아닌 것 (제거 대상)
# 1. 키워드 정의
# 오피오이드 성분명 (이 단어가 들어가면 무조건 오피오이드로 인정)
opioid_whitelist = [
    'fentanyl', 'morphine', 'oxycodone', 'hydromorphone', 'methadone', 
    'hydrocodone', 'tramadol', 'meperidine', 'codeine', 'buprenorphine', 
    'tapentadol', 'oxymorphone', 'levorphanol', 'remifentanil', 'sufentanil',
    'dilaudid', 'demerol', 'ultram', 'vicodin', 'percocet', 'oxycontin',
    'opium' # opium tincture 등을 잡기 위함
]

# 제거해야 할 비오피오이드 및 노이즈 키워드
fp_keywords = [
    'norepinephrine', 'guaifenesin', 'nuedexta', 'acetaminophen', 
    'rob', '1', '2', 'respiratory', 'vaccine'
]

# ---------------------------------------------------------
# Step 1: FP 탐지 및 제거 (cancer_opioid 에서)
# ---------------------------------------------------------

def is_real_false_positive(drug_name):
    if pd.isna(drug_name): return False
    d_lower = drug_name.lower()
    
    # [중요] 1. 오피오이드 성분명이 있으면 절대 FP가 아님 (살려둠)
    # 예: Oxycodone-Acetaminophen 은 여기서 걸러져서 False 반환됨
    for op in opioid_whitelist:
        if op in d_lower:
            return False 
            
    # 2. 오피오이드 성분이 없는 것들 중, FP 키워드가 있으면 제거 대상
    for kw in fp_keywords:
        if kw in d_lower:
            return True
            
    return False

# FP 식별
fp_mask = cancer_opioid['drug'].apply(is_real_false_positive)
fp_rows = cancer_opioid[fp_mask].copy()

print(f"Detected False Positives (FP): {len(fp_rows)} rows")
# 이제 Oxycodone 등은 안 보여야 하고, Norepinephrine만 보여야 함
if not fp_rows.empty:
    print(fp_rows[['subject_id', 'hadm_id', 'ndc', 'drug']].head(5))

# FP 제거 적용
cancer_opioid_cleaned = cancer_opioid[~fp_mask].copy()

# ---------------------------------------------------------
# Step 2: FN 탐지 및 추가 (cancer_non_opioid 대상자들의 전체 처방 검색)
# ---------------------------------------------------------

# ---------------------------------------------------------
# Step 2: FN 탐지 및 추가 (기존 로직 유지)
# ---------------------------------------------------------
# cancer_non_opioid 대상자들의 hadm_id
non_opioid_hadm_ids = cancer_non_opioid['hadm_id'].unique().tolist()

# SQL 파라미터 개수 제한 방지를 위해 청크(Chunk) 단위로 조회하거나
# 여기서는 pandas read_sql 로직 유지 (데이터 양에 따라 주의)
if non_opioid_hadm_ids:
    placeholders = ','.join(['?'] * len(non_opioid_hadm_ids))
    fn_query = f"""
    SELECT subject_id, hadm_id, ndc, drug
    FROM prescriptions
    WHERE hadm_id IN ({placeholders})
    """
    try:
        potential_fn_prescriptions = pd.read_sql_query(fn_query, conn, params=non_opioid_hadm_ids)
    except Exception as e:
        print("SQL Error (Likely too many parameters). Try chunking list.")
        potential_fn_prescriptions = pd.DataFrame()
else:
    potential_fn_prescriptions = pd.DataFrame()

# 1. 키워드 정의
# 오피오이드 성분명 (유지)


# [수정됨] FN 탐지 시 제외해야 할 '가짜 opium' 키워드 추가
# 호흡기 약물들이 여기서 걸러집니다.
fn_exclusion_keywords = [
    'ipratropium', 'tiotropium', 'tropium', # ~tropium 계열 (기관지 확장제)
    'loperamide', # 지사제 (오피오이드 수용체에 작용하지만 중추신경 효과 없음, 진통제 아님)
    'dextromethorphan' # 진해제 (오피오이드 구조지만 진통 효과 미미/비마약성 분류)
]

def is_false_negative_refined(drug_name):
    if pd.isna(drug_name): return False
    d_lower = drug_name.lower()
    
    # 1. 먼저 제외 키워드가 있는지 확인 (Safety Check)
    for ex in fn_exclusion_keywords:
        if ex in d_lower:
            return False # 오피오이드 아님 (Ipratropium 등 탈락)

    # 2. 오피오이드 키워드가 포함되어 있는지 확인
    for kw in opioid_whitelist:
        if kw in d_lower:
            # 안전장치 재확인: 노르에피네프린 등 제외
            if 'norepinephrine' in d_lower: return False
            return True
            
    return False
# FN 식별 (수정된 함수 사용)
fn_mask = potential_fn_prescriptions['drug'].apply(is_false_negative_refined)
fn_rows = potential_fn_prescriptions[fn_mask].copy()

# 결과 확인
fn_ndc_summary = fn_rows.groupby(['ndc', 'drug']).size().reset_index(name='count')
fn_ndc_summary = fn_ndc_summary.sort_values(by='count', ascending=False)

print(f"--- [수정 후] 새롭게 찾아낸(FN) 고유 NDC 개수: {len(fn_ndc_summary)}개 ---")
print(fn_ndc_summary.head(20)) # 상위 20개 다시 확인

# 원본 cancer_admissions 정보와 병합
if not fn_rows.empty:
    fn_rows_merged = pd.merge(fn_rows, cancer_admissions[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'], how='inner')
else:
    fn_rows_merged = pd.DataFrame()


# ---------------------------------------------------------
# Step 3: Final Dataframe 생성
# ---------------------------------------------------------
# Final Opioid 병합
final_cancer_opioid = pd.concat([cancer_opioid_cleaned, fn_rows_merged], ignore_index=True)
final_cancer_opioid = final_cancer_opioid.drop_duplicates(subset=['subject_id', 'hadm_id', 'ndc', 'drug'])

# Final Non-Opioid 계산
opioid_hadm_set = set(final_cancer_opioid['hadm_id'].unique())
final_cancer_non_opioid = cancer_admissions[~cancer_admissions['hadm_id'].isin(opioid_hadm_set)].copy()

print(f"\n[Result]")
print(f"Final Cancer Opioid Admissions: {final_cancer_opioid['hadm_id'].nunique()}")
print(f"Final Cancer Non-Opioid Admissions: {len(final_cancer_non_opioid)}")

print("\n[Check Corrected FP Logic]")
print("Excluded Items (Should ONLY contain Norepinephrine, Guaifenesin-only, etc):")
if not fp_rows.empty:
    print(fp_rows[['drug']].value_counts().head(10))
    print(fp_rows[['ndc']].value_counts().head(10))
conn.close()


--- 사후 필터링 및 재분류 시작 ---

Detected False Positives (FP): 740 rows
     subject_id   hadm_id          ndc            drug
40     10003019  22774359  61553015311  NORepinephrine
176    10014078  25809882  61553015311  NORepinephrine
278    10020944  29974575  61553015311  NORepinephrine
366    10026161  24614671  61553015311  NORepinephrine
858    10072153  21253864  61553015311  NORepinephrine
--- [수정 후] 새롭게 찾아낸(FN) 고유 NDC 개수: 122개 ---
             ndc                                       drug  count
58   00409176230                           Morphine Sulfate   1773
60   00409189001                           Morphine Sulfate   1380
7    00054023524                        Morphine Sulfate IR   1214
27   00054839224                   HYDROmorphone (Dilaudid)    856
74   10019017644                           Morphine Sulfate    708
64   00641607325                           Morphine Sulfate    592
56   00409125830                           Morphine Sulfate    565
65   00641612525         

In [14]:
# FN 데이터에서 NDC와 약물명으로 그룹화하여 빈도수 계산
fn_ndc_summary = fn_rows.groupby(['ndc', 'drug']).size().reset_index(name='count')

# 빈도수가 높은 순서대로 정렬 (가장 많이 놓친 약물 확인)
fn_ndc_summary = fn_ndc_summary.sort_values(by='count', ascending=False)

print(f"--- 새롭게 찾아낸(FN) 고유 NDC 개수: {len(fn_ndc_summary)}개 ---")
print(fn_ndc_summary.head(30))  # 상위 30개 출력

# (선택사항) 이 리스트를 파일로 저장해서 나중에 MME 테이블 업데이트에 활용
# fn_ndc_summary.to_csv("missed_opioid_ndcs.csv", index=False)

--- 새롭게 찾아낸(FN) 고유 NDC 개수: 122개 ---
             ndc                                       drug  count
58   00409176230                           Morphine Sulfate   1773
60   00409189001                           Morphine Sulfate   1380
7    00054023524                        Morphine Sulfate IR   1214
27   00054839224                   HYDROmorphone (Dilaudid)    856
74   10019017644                           Morphine Sulfate    708
64   00641607325                           Morphine Sulfate    592
56   00409125830                           Morphine Sulfate    565
65   00641612525                           Morphine Sulfate    562
52   00406833062                    Morphine SR (MS Contin)    501
50   00406831562                    Morphine SR (MS Contin)    419
42   00074241612                   HYDROmorphone (Dilaudid)    300
62   00409606211                           Morphine Sulfate    265
15   00054023755              Morphine Sulfate (Oral Soln.)    264
40   00074233411          

In [15]:
import os

# 저장 경로 설정 (사용자 경로 유지)
SAVE_DIR = r"C:\Users\kevib\dev\DA\team"

# 1. FP (False Positive) NDC 저장
# - 설명: 오피오이드로 잘못 분류되었던 비마약성 약물들 (제거 대상)
if not fp_rows.empty:
    unique_fp_ndc = fp_rows.groupby(['ndc', 'drug']).size().reset_index(name='count')
    unique_fp_ndc = unique_fp_ndc.sort_values(by='count', ascending=False)
    
    fp_save_path = os.path.join(SAVE_DIR, "detected_FP_ndcs_to_remove.csv")
    unique_fp_ndc.to_csv(fp_save_path, index=False)
    print(f"✅ FP NDC 목록 저장 완료: {fp_save_path} (총 {len(unique_fp_ndc)}개)")
else:
    print("⚠️ 저장할 FP 데이터가 없습니다.")

# 2. FN (False Negative) NDC 저장
# - 설명: 기존 리스트에 없어서 새로 찾아낸 오피오이드 약물들 (추가 대상)
if not fn_rows.empty:
    unique_fn_ndc = fn_rows.groupby(['ndc', 'drug']).size().reset_index(name='count')
    unique_fn_ndc = unique_fn_ndc.sort_values(by='count', ascending=False)
    
    fn_save_path = os.path.join(SAVE_DIR, "detected_FN_ndcs_to_add.csv")
    unique_fn_ndc.to_csv(fn_save_path, index=False)
    print(f"✅ FN NDC 목록 저장 완료: {fn_save_path} (총 {len(unique_fn_ndc)}개)")
else:
    print("⚠️ 저장할 FN 데이터가 없습니다.")

✅ FP NDC 목록 저장 완료: C:\Users\kevib\dev\DA\team\detected_FP_ndcs_to_remove.csv (총 7개)
✅ FN NDC 목록 저장 완료: C:\Users\kevib\dev\DA\team\detected_FN_ndcs_to_add.csv (총 122개)


In [16]:
import sqlite3
import pandas as pd

# DB 경로 (사용자 설정 유지)
DB_PATH = r"C:\Users\kevib\dev\DA\team\MIMIC4-hosp-icu.db"
conn = sqlite3.connect(DB_PATH)

print("--- 데이터베이스 테이블 생성 시작 ---")

# 1. 오피오이드 처방 테이블 생성 (final_cancer_opioid_admissions)
# - 내용: 암 환자 중 오피오이드 처방이 있는 건들의 상세 내역 (subject_id, hadm_id, ndc, drug 등)
final_cancer_opioid.to_sql(
    name='final_cancer_opioid_admissions', 
    con=conn, 
    if_exists='replace',  # 이미 테이블이 있다면 덮어쓰기
    index=False           # pandas index는 저장 안 함
)
print(f"✅ 'final_cancer_opioid_admissions' 테이블 생성 완료 (행 개수: {len(final_cancer_opioid)})")

# 2. 비오피오이드 대조군 테이블 생성 (final_cancer_non_opioid_admissions)
# - 내용: 암 환자 중 오피오이드 처방이 '전혀' 없는 입원 건 (subject_id, hadm_id)
# - 주의: 이 DF에는 처방 정보(ndc, drug)가 없고 입원 정보만 있을 것입니다.
final_cancer_non_opioid.to_sql(
    name='final_cancer_non_opioid_admissions', 
    con=conn, 
    if_exists='replace', 
    index=False
)
print(f"✅ 'final_cancer_non_opioid_admissions' 테이블 생성 완료 (행 개수: {len(final_cancer_non_opioid)})")


# 3. 조회 속도 향상을 위한 인덱스 생성 (선택 사항이지만 강력 권장)
cursor = conn.cursor()

# 오피오이드 테이블에 hadm_id, ndc 인덱스 추가
cursor.execute("CREATE INDEX IF NOT EXISTS idx_opioid_hadm ON final_cancer_opioid_admissions (hadm_id)")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_opioid_ndc ON final_cancer_opioid_admissions (ndc)")

# 비오피오이드 테이블에 hadm_id 인덱스 추가
cursor.execute("CREATE INDEX IF NOT EXISTS idx_non_opioid_hadm ON final_cancer_non_opioid_admissions (hadm_id)")

print("✅ 인덱스(Index) 생성 완료 (검색 속도 최적화)")

conn.commit()
conn.close()
print("\n--- 모든 작업 완료 ---")

--- 데이터베이스 테이블 생성 시작 ---
✅ 'final_cancer_opioid_admissions' 테이블 생성 완료 (행 개수: 139488)
✅ 'final_cancer_non_opioid_admissions' 테이블 생성 완료 (행 개수: 29823)
✅ 인덱스(Index) 생성 완료 (검색 속도 최적화)

--- 모든 작업 완료 ---


In [17]:
import pandas as pd
import sqlite3

# DB 연결
DB_PATH = r"C:\Users\kevib\dev\DA\team\MIMIC4-hosp-icu.db"
conn = sqlite3.connect(DB_PATH)

print("--- [최종 검증] 데이터 무결성 테스트 시작 ---\n")

# 1. DB에서 저장된 테이블의 hadm_id 가져오기
df_op_db = pd.read_sql("SELECT DISTINCT hadm_id FROM final_cancer_opioid_admissions", conn)
df_non_db = pd.read_sql("SELECT DISTINCT hadm_id FROM final_cancer_non_opioid_admissions", conn)

# 집합(Set)으로 변환
set_opioid = set(df_op_db['hadm_id'])
set_non_opioid = set(df_non_db['hadm_id'])

# 원본 암 환자 모집단 (cancer_admissions 변수가 메모리에 있다고 가정)
# 만약 없다면 다시 로드 필요
set_total_cancer = set(cancer_admissions['hadm_id'])

# -----------------------------------------------------------
# 검증 1: 교집합 확인 (상호 배타성)
# "한 환자의 입원이 오피오이드 그룹이면서 동시에 비오피오이드 그룹일 수 없다."
# -----------------------------------------------------------
intersection = set_opioid.intersection(set_non_opioid)
print(f"1. 교집합(중복) 검증: {'✅ 통과 (PASS)' if len(intersection) == 0 else '❌ 실패 (FAIL)'}")

if len(intersection) > 0:
    print(f"   ⚠️ 경고: {len(intersection)}건의 hadm_id가 두 테이블 모두에 존재합니다.")
    print(f"   Sample: {list(intersection)[:5]}")
else:
    print("   -> 두 그룹 간에 겹치는 hadm_id가 하나도 없습니다.")


# -----------------------------------------------------------
# 검증 2: 합집합 확인 (완전성)
# "오피오이드 그룹 + 비오피오이드 그룹 = 전체 암 환자 모집단이어야 한다."
# -----------------------------------------------------------
union_set = set_opioid.union(set_non_opioid)

# 2-1. 누락된 데이터 확인 (Total - Union)
missing_ids = set_total_cancer - union_set

# 2-2. 뜬금없이 생긴 데이터 확인 (Union - Total)
unexpected_ids = union_set - set_total_cancer

is_complete = (len(missing_ids) == 0) and (len(unexpected_ids) == 0)

print(f"\n2. 합집합(완전성) 검증: {'✅ 통과 (PASS)' if is_complete else '❌ 실패 (FAIL)'}")

if len(missing_ids) > 0:
    print(f"   ⚠️ 누락됨(Missing): {len(missing_ids)}건의 hadm_id가 분류 과정에서 사라졌습니다.")
elif len(unexpected_ids) > 0:
    print(f"   ⚠️ 초과됨(Unexpected): {len(unexpected_ids)}건의 hadm_id가 원본 모집단에 없는데 생성되었습니다.")
else:
    print("   -> 두 그룹의 합이 원본 암 환자 리스트와 정확히 일치합니다.")

print(f"\n[상세 수치]")
print(f"- 오피오이드 그룹: {len(set_opioid):,} 명")
print(f"- 비오피오이드 그룹: {len(set_non_opioid):,} 명")
print(f"- 합계: {len(union_set):,} 명")
print(f"- 원본(Target): {len(set_total_cancer):,} 명")

conn.close()

--- [최종 검증] 데이터 무결성 테스트 시작 ---

1. 교집합(중복) 검증: ✅ 통과 (PASS)
   -> 두 그룹 간에 겹치는 hadm_id가 하나도 없습니다.

2. 합집합(완전성) 검증: ✅ 통과 (PASS)
   -> 두 그룹의 합이 원본 암 환자 리스트와 정확히 일치합니다.

[상세 수치]
- 오피오이드 그룹: 67,267 명
- 비오피오이드 그룹: 29,823 명
- 합계: 97,090 명
- 원본(Target): 97,090 명
